In [1]:
# Code from: https://github.com/gokriznastic/HybridSN
import keras
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

import mypackage

init_notebook_mode(connected=True)
%matplotlib inline

Using TensorFlow backend.


TensorFlow version is 2.1.0


In [2]:
%load_ext autoreload
%autoreload 2

import mypackage

# Data Loading

In [3]:
## GLOBAL VARIABLES
dataset = 'HyperChicken'
test_ratio = 0.3
windowSize = 100

In [4]:
def loadData(name):
    if name == 'HyperChicken':
        data, labels, info = mypackage.Dataset.load("../../Code/data/tomra")
        labels -= 1
    return data, labels

In [5]:
X, y = loadData(dataset)

X.shape, y.shape

((34, 100, 100, 208), (34, 100, 100, 1))

In [6]:
np.unique(y)

array([0, 1, 2], dtype=int32)

In [7]:
Xtrain, Xtest, ytrain, ytest = mypackage.Dataset.train_test_split(X, y, test_ratio)
# ytrain, ytest = ytrain.ravel(), ytest.ravel()

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((23, 100, 100, 208),
 (11, 100, 100, 208),
 (23, 100, 100, 1),
 (11, 100, 100, 1))

In [8]:
Xtrain, Xtest, ytrain, ytest = mypackage.Dataset.train_test_split(X, y, test_ratio)
ytrain, ytest = np.squeeze(ytrain), np.squeeze(ytest)

Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape

((23, 100, 100, 208), (11, 100, 100, 208), (23, 100, 100), (11, 100, 100))

In [9]:
K = 30
Xtrain, Xtest = mypackage.Dataset.PCA(Xtrain, Xtest, n_components=K)

Xtrain.shape

(23, 100, 100, 30)

Xtrain, Xvalid, ytrain, yvalid = splitTrainTestSet(Xtrain, ytrain, 0.3333)

Xtrain.shape, Xvalid.shape, ytrain.shape, yvalid.shape

# Model and Training

In [10]:
Xtrain = Xtrain.reshape(-1, windowSize, windowSize, K, 1)
Xtrain.shape

(23, 100, 100, 30, 1)

In [11]:
np.unique(ytrain)

array([0, 1, 2], dtype=int32)

In [12]:
ytrain = np_utils.to_categorical(ytrain)
ytrain.shape

(23, 100, 100, 3)

Xvalid = Xvalid.reshape(-1, windowSize, windowSize, K, 1)
Xvalid.shape

yvalid = np_utils.to_categorical(yvalid)
yvalid.shape

In [13]:
S = windowSize
L = K
output_units = ytrain.shape[-1]

## TODO:
Add the upsampling

https://towardsdatascience.com/understanding-semantic-segmentation-with-unet-6be4f42d4b47

https://github.com/hlamba28/UNET-TGS/blob/master/TGS%20UNET.ipynb

###########################

https://github.com/zhixuhao/unet

https://github.com/karolzak/keras-unet

https://www.kaggle.com/krishanudb/keras-based-unet-model-construction-tutorial

### Kernel size
The first kernel depth shoud probably cover close to 100nm, which is about 28 channels. Maybe 50nm is sufficient, that is about 16 channels.

In [14]:
## input layer
input_layer = Input((S, S, L, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu')(conv_layer1)
conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(conv_layer2)
print(conv_layer3._keras_shape)
conv3d_shape = conv_layer3._keras_shape
conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu')(conv_layer3)

flatten_layer = Flatten()(conv_layer4)

## fully connected layers
dense_layer1 = Dense(units=256, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=128, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=output_units, activation='softmax')(dense_layer2)

(None, 94, 94, 18, 32)


In [15]:
# define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

In [16]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 30, 1)   0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 98, 98, 24, 8)     512       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 96, 96, 20, 16)    5776      
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 94, 94, 18, 32)    13856     
_________________________________________________________________
reshape_1 (Reshape)          (None, 94, 94, 576)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 92, 92, 64)        331840    
_________________________________________________________________
flatten_1 (Flatten)          (None, 541696)            0   

In [17]:
# compiling the model
adam = Adam(lr=0.001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [18]:
# checkpoint
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [19]:
Xtrain.shape, ytrain.shape

((23, 100, 100, 30, 1), (23, 100, 100, 3))

In [20]:
# import matplotlib.pyplot as plt
# plt.imshow(ytrain[0])

In [21]:
# Had batch_size=256
history = model.fit(x=Xtrain, y=ytrain, batch_size=50, epochs=2, callbacks=callbacks_list)

ValueError: Error when checking target: expected dense_3 to have 2 dimensions, but got array with shape (23, 100, 100, 3)

plt.figure(figsize=(7,7))
plt.grid()
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'], loc='upper right')
plt.savefig("loss_curve.pdf")
plt.show()

plt.figure(figsize=(5,5))
plt.ylim(0,1.1)
plt.grid()
plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Training','Validation'])
plt.savefig("acc_curve.pdf")
plt.show()

# Validation

In [ ]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
Xtest = Xtest.reshape(-1, windowSize, windowSize, K, 1)
Xtest.shape

In [ ]:
ytest = np_utils.to_categorical(ytest)
ytest.shape

In [ ]:
Y_pred_test = model.predict(Xtest)
y_pred_test = np.argmax(Y_pred_test, axis=1)

classification = classification_report(np.argmax(ytest, axis=1), y_pred_test)
print(classification)

In [ ]:
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [ ]:
def reports (X_test,y_test,name):
    #start = time.time()
    Y_pred = model.predict(X_test)
    y_pred = np.argmax(Y_pred, axis=1)
    #end = time.time()
    #print(end - start)
    if name == 'IP':
        target_names = ['Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn'
                        ,'Grass-pasture', 'Grass-trees', 'Grass-pasture-mowed', 
                        'Hay-windrowed', 'Oats', 'Soybean-notill', 'Soybean-mintill',
                        'Soybean-clean', 'Wheat', 'Woods', 'Buildings-Grass-Trees-Drives',
                        'Stone-Steel-Towers']
    elif name == 'SA':
        target_names = ['Brocoli_green_weeds_1','Brocoli_green_weeds_2','Fallow','Fallow_rough_plow','Fallow_smooth',
                        'Stubble','Celery','Grapes_untrained','Soil_vinyard_develop','Corn_senesced_green_weeds',
                        'Lettuce_romaine_4wk','Lettuce_romaine_5wk','Lettuce_romaine_6wk','Lettuce_romaine_7wk',
                        'Vinyard_untrained','Vinyard_vertical_trellis']
    elif name == 'PU':
        target_names = ['Asphalt','Meadows','Gravel','Trees', 'Painted metal sheets','Bare Soil','Bitumen',
                        'Self-Blocking Bricks','Shadows']
    elif name == 'HyperChicken':
        target_names = ['Belt','Meat','Plastic']
    
    classification = classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names)
    oa = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    confusion = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(np.argmax(y_test, axis=1), y_pred)
    score = model.evaluate(X_test, y_test, batch_size=32)
    Test_Loss =  score[0]*100
    Test_accuracy = score[1]*100
    
    return classification, confusion, Test_Loss, Test_accuracy, oa*100, each_acc*100, aa*100, kappa*100

In [ ]:
classification, confusion, Test_loss, Test_accuracy, oa, each_acc, aa, kappa = reports(Xtest,ytest,dataset)
classification = str(classification)
confusion = str(confusion)
file_name = "classification_report.txt"

with open(file_name, 'w') as x_file:
    x_file.write('{} Test loss (%)'.format(Test_loss))
    x_file.write('\n')
    x_file.write('{} Test accuracy (%)'.format(Test_accuracy))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

In [ ]:
def Patch(data,height_index,width_index):
    height_slice = slice(height_index, height_index+PATCH_SIZE)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    patch = data[height_slice, width_slice, :]
    
    return patch

In [ ]:
# load the original image
X, y = loadData(dataset)

In [ ]:
height = y.shape[0]
width = y.shape[1]
PATCH_SIZE = windowSize
numComponents = K

In [ ]:
X,pca = applyPCA(X, numComponents=numComponents)

In [ ]:
X = padWithZeros(X, PATCH_SIZE//2)

In [ ]:
# calculate the predicted image
outputs = np.zeros((height,width))
for i in range(height):
    for j in range(width):
        target = int(y[i,j])
        if target == 0 :
            continue
        else :
            image_patch=Patch(X,i,j)
            X_test_image = image_patch.reshape(1,image_patch.shape[0],image_patch.shape[1], image_patch.shape[2], 1).astype('float32')                                   
            prediction = (model.predict(X_test_image))
            prediction = np.argmax(prediction, axis=1)
            outputs[i][j] = prediction+1

In [ ]:
ground_truth = spectral.imshow(classes = np.squeeze(y),figsize =(7,7))

In [ ]:
predict_image = spectral.imshow(classes = outputs.astype(int),figsize =(7,7))

In [ ]:
# spectral.save_rgb("predictions.jpg", outputs.astype(int), colors=spectral.spy_colors)

spectral.save_rgb(str(dataset)+"_ground_truth.jpg", y, colors=spectral.spy_colors)